<a href="https://colab.research.google.com/github/royder789/Vgg16/blob/main/vgg16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import torch.nn as nn#neural network
from torchvision import datasets #cifar10,cifar100
from torchvision import transforms# to chmage np array to tensor
from torch.utils.data.sampler import SubsetRandomSampler



In [ ]:
#devuice configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [ ]:
#data loader
def data_loader(data_dir,batch_size, random_seed=42,shuffle=True,valid_size=0.1,test=False):
  #for normalising everything thorugh std and mean
  normalize=transforms.Normalize(
      mean=[0.4914,0.4822,0.4465],#x-mean/std
      std=[0.2023,0.1994,0.2010]
  )
  #define transfoms
  transform = transforms.Compose([
      transforms.Resize((227,227)),
      transforms.ToTensor(),
      normalize
  ])


  if test:
    dataset = datasets.CIFAR100(
        root=data_dir,train=False,download=True,transform=transform
    )

    data_loader = torch.utils.data.DataLoader(
        dataset,batch_size=batch_size,shuffle=shuffle
    )
    return data_loader


  #load the dataset
  train_dataset = datasets.CIFAR100(
      root=data_dir,train=True,download=True,transform=transform
  )

  #validation dataset  during trainng time
  valid_dataset = datasets.CIFAR100(
      root=data_dir,train=True,download=True,transform=transform
  )

  num_train = len(train_dataset)
  indices = list(range(num_train))
  split = int(np.floor(valid_size*num_train))

  if shuffle:
    np.random.seed(random_seed)
    np.random.shuffle(indices)

  train_idx,valid_idx = indices[split:],indices[:split]
  train_sampler = SubsetRandomSampler(train_idx)
  valid_sampler = SubsetRandomSampler(valid_idx)

  train_loader = torch.utils.data.DataLoader(
      train_dataset,batch_size=batch_size,sampler=train_sampler
  )

  valid_loader = torch.utils.data.DataLoader(
      valid_dataset,batch_size=batch_size,sampler=valid_sampler
  )

  return (train_loader,valid_loader)



In [ ]:
#calling function
train_loader,valid_laoder=data_loader(data_dir='./data',batch_size=64)

100%|██████████| 169M/169M [00:02<00:00, 57.3MB/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
#when for testing just make it true
test_loader=data_loader(data_dir='./data',batch_size=64,test=True)


Files already downloaded and verified


okay so all the dta loading part is done now we will start the main part for vgg16

In [ ]:

class VGG16(nn.Module):
    def __init__(self,n_classes):
        super(VGG16, self).__init__()
        self.layer1=nn.Sequential(
            nn.Conv2d(3,64,kernel_size=3,stride=1,padding=1),
            #as vgg didnt want to reduce of size of images henc ewe will do teh padding
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.layer2=nn.Sequential(
            #(64,64)first the output ferom previous then enxt
            nn.Conv2d(64,64,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.layer3=nn.Sequential(
            nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
            #as vgg didnt want to reduce of size of images henc ewe will do teh padding
            nn.BatchNorm2d(128),
            nn.ReLU()
        )
        self.layer4=nn.Sequential(
            #(64,64)first the output ferom previous then enxt
            nn.Conv2d(128,128,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.layer5=nn.Sequential(
            #(64,64)first the output ferom previous then enxt
            nn.Conv2d(128,256,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU()

        )
        self.layer6=nn.Sequential(
            #(64,64)first the output ferom previous then enxt
            nn.Conv2d(256,256,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU()

        )
        self.layer7=nn.Sequential(
            #(64,64)first the output ferom previous then enxt
            nn.Conv2d(256,256,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.layer8=nn.Sequential(
            #(64,64)first the output ferom previous then enxt
            nn.Conv2d(256,512,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU()

        )
        self.layer9=nn.Sequential(
            #(64,64)first the output ferom previous then enxt
            nn.Conv2d(512,512,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU()

        )
        self.layer10=nn.Sequential(
            #(64,64)first the output ferom previous then enxt
            nn.Conv2d(512,512,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.layer11=nn.Sequential(
            #(64,64)first the output ferom previous then enxt
            nn.Conv2d(512,512,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU()

        )
        self.layer12=nn.Sequential(
            #(64,64)first the output ferom previous then enxt
            nn.Conv2d(512,512,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU()

        )
        self.layer13=nn.Sequential(
            #(64,64)first the output ferom previous then enxt
            nn.Conv2d(512,512,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
#size=n-2p+k/s+1
#n=227,p=padding=1,k=kernelsize=3,s=strude=1
#227-2*1+3/1+1
#229*229*64



        #fully connceted layer
        self.fc=nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(7*7*512,4096),
            nn.ReLU()
        )
        self.fc1=nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096,4096),
            nn.ReLU()
        )
        self.fc2=nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096,n_classes),
            nn.ReLU()
        )

    def forward(self, x):
        out = self.layer1(x)  # Process input through the first layer
        out = self.layer2(out)  # Pass output to the next layer
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = self.layer9(out)
        out = self.layer10(out)
        out = self.layer11(out)
        out = self.layer12(out)
        out = self.layer13(out)
        # Flatten for the fully connected layers
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out












In [ ]:
"""def vgg16(x, n_classes):

    layer1 = nn.Sequential(
        nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU()
    )
    x = layer1(x)
    layer2 = nn.Sequential(
        nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    x = layer2(x)
    layer3 = nn.Sequential(
        nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU()
    )
    x = layer3(x)
    layer4 = nn.Sequential(
        nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    x = layer4(x)
    layer5 = nn.Sequential(
        nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU()
    )
    x = layer5(x)
    layer6 = nn.Sequential(
        nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU()
    )
    x = layer6(x)
    layer7 = nn.Sequential(
        nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    x = layer7(x)
    layer8 = nn.Sequential(
        nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(512),
        nn.ReLU()
    )
    x = layer8(x)
    layer9 = nn.Sequential(
        nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(512),
        nn.ReLU()
    )
    x = layer9(x)
    layer10 = nn.Sequential(
        nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    x = layer10(x)
    layer11 = nn.Sequential(
        nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(512),
        nn.ReLU()
    )
    x = layer11(x)
    layer12 = nn.Sequential(
        nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(512),
        nn.ReLU()
    )
    x = layer12(x)
    layer13 = nn.Sequential(
        nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    x = layer13(x)
    x = x.view(x.size(0), -1)

    fc = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(7 * 7 * 512, 4096),
        nn.ReLU()
    )
    x = fc(x)

    fc1 = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(4096, 4096),
        nn.ReLU()
    )
    x = fc1(x)

    fc2 = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(4096, n_classes)
    )
    x = fc2(x)

    return x"""

In [ ]:
n_classes=100
num_epochs=9
batch_size=16
learning_rate=0.001

In [ ]:

model=VGG16(n_classes).to(device)

In [ ]:
model.parameters

<bound method Module.parameters of VGG16(
  (layer1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer4): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer5): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer6): Sequential(
    (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer7): Sequential(
    (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer8): Sequential(
    (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer9): Sequential(
    (0): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer10): Sequential(
    (0): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer11): Sequential(
    (0): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer12): Sequential(
    (0): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer13): Sequential(
    (0): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=25088, out_features=4096, bias=True)
    (2): ReLU()
  )
  (fc1): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=4096, out_features=4096, bias=True)
    (2): ReLU()
  )
  (fc2): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=4096, out_features=100, bias=True)
    (2): ReLU()
  )
)>

In [ ]:
criteria=nn.CrossEntropyLoss( )
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)

In [ ]:
total_step=len(train_loader)
total_step
for images, labels in train_loader:
    print(images.shape)  # Should output: torch.Size([batch_size, 3, 227, 227])
    break


torch.Size([64, 3, 227, 227])


In [ ]:
# Training loop
for epoch in range(num_epochs):

    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criteria(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

# Validation loop

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in valid_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Validation Accuracy: {accuracy:.2f}%')


Epoch [1/9], Step [1/704], Loss: 4.6529
Epoch [1/9], Step [2/704], Loss: 13.3262


KeyboardInterrupt: 